# Integrating Attribute and Boundary Data From Different Sources

[Serge Rey](http://sergerey.org)

A common workflow in GIS in the social science is to combine attribute data from one source with boundaries for the administrative units of interest that are obtained from a different source. To work through an example of this work flow we will consider two sources of data:

* California Per Capita Income Data (2015, 2016, 2017) [(Source)](https://www.bea.gov/system/files/2018-11/lapi1118_0.xlsx)
* California County Shapefile [(Source)](https://data.ca.gov/sites/default/files/CA_Counties_0.zip)

After downloading and extracting the files, the spreadsheet for California County Income was constructed by selecting out counties for California from the national file. The resulting file resides in `data/ca_county_pcincome.xlxs`.

Let's create a pandas dataframe by reading this spreadsheet:

In [ ]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sbn

%matplotlib  inline

In [ ]:
pc_income_df = pd.read_excel('data/ca_county_pcincome.xlsx')

In [ ]:
pc_income_df.head()

In [ ]:
h = sbn.distplot(pc_income_df['PCI2015'], bins=5, rug=True);

### Sorting

In [ ]:
pc_income_df.sort_values(by='PCI2015').head()

In [ ]:
pc_income_df.sort_values(by='PCI2015', ascending=False).head()

In [ ]:
pc_income_df.sort_values(by='PCI2017').head()

In [ ]:
pc_income_df.sort_values(by='PCI2017', ascending=False).head()

## Boundaries

The geometries for the counties are in `data/CA_Counties_TIGER2016.shp`. 

Using geopandas, we can read these into a GeoDataFrame:

In [ ]:
gdf = gpd.read_file('data/CA_Counties_TIGER2016.shp')

In [ ]:
gdf.plot()

In [ ]:
gdf.head()

## Table Join

Examination of the GeoDataFrame shows us many attributes of the counties, but the income data are not among those. 
We need to integrate the income data from the pandas DataFrame into the GeoDataFrame.

To do this we rely on what is known as a **table join**.

We need to find a pair of attributes, one from the pandas DataFrame and one from the GeoDataFrame, that have values we can align. In other words, those attributes should measure the same thing, and have the same value for the record for each county (and that has to be unique to the county). 

Examination of the two sets of columns suggests that using the county names might be a good way to go. The one complication is that in the DataFrame, the names of the county are in the attribute `County`, while in the GeoDataFrame they are in the attribute `NAME`.

In Geopandas a table join is done using the `merge` method. We will call the `merge` method on the GeoDataFrame (which will be referred to as the "left" table, to merge on the `County` attribute of the income dataframe (which is the "right" table):

In [ ]:
jdf = gdf.merge(pc_income_df, left_on='NAME', right_on='County')

In [ ]:
jdf.head()

In [ ]:
jdf.columns

In [ ]:
gdf.columns

In [ ]:
gdf.merge?

In [ ]:
jdf.plot(column='PCI2015')

## Using Choropleth Helper

In [ ]:
from choropleth import choropleth

j = jdf # for the lazy professor

In [ ]:
choropleth(j, 'PCI2015',
           scheme='equal_interval', k=5)

## Fix Legend

In [ ]:
choropleth(j, 'PCI2015',
           scheme='equal_interval', k=5, label_int=True)

## Changing the Size of the Map

In [ ]:
f, ax = plt.subplots(1, figsize=(12, 8))
ax = choropleth(j, 'PCI2015',
           scheme='equal_interval', k=5, label_int=True, edgecolor='black', ax=ax)

In [ ]:
choropleth(j, 'PCI2015', cmap='Blues',
           scheme='fisher_jenks', k=5)

In [ ]:
choropleth(j, 'PCI2015', cmap='Blues',
           scheme='quantiles', k=5, label_int=True)

In [ ]:
choropleth(j, 'PCI2015', cmap='Blues',
           scheme='quantiles', k=5, label_int=True)

In [ ]:
choropleth(j, 'PCI2015', cmap='Blues',
           scheme='quantiles', k=5, label_int=True, edgecolor='black')

In [ ]:
choropleth(j, 'PCI2015', cmap='Blues',
           scheme='fisher_jenks', k=5, label_int=True, edgecolor='black')

In [ ]:
f, ax = plt.subplots(1, figsize=(12, 12))
ax = choropleth(j, 'PCI2015', cmap='Blues',
           scheme='fisher_jenks', k=5, label_int=True, edgecolor='black', ax=ax)
plt.axis('off')

## Save our joined gdf

In [ ]:
j.to_file('data/pcigdf.shp')

In [ ]:
f, ax = plt.subplots(1, figsize=(12, 12))
ax = choropleth(j, 'PCI2015', cmap='Blues',
           scheme='fisher_jenks', k=5, label_int=True, edgecolor='black', ax=ax)
plt.axis('off')
plt.title('PCI 2015')
plt.savefig('pci2015.png')

## Labeling

In [ ]:
ax = j.plot()
j.apply(lambda x: ax.annotate(s=x.NAME, xy=x.geometry.centroid.coords[0], ha='center'),axis=1);

In [ ]:
f, ax = plt.subplots(1, figsize=(12, 12))
ax = choropleth(j, 'PCI2015', cmap='Blues',
           scheme='fisher_jenks', k=5, label_int=True, edgecolor='black', ax=ax)
j.apply(lambda x: ax.annotate(s=x.NAME, xy=x.geometry.centroid.coords[0], ha='center'),axis=1);
plt.axis('off')
plt.title('PCI 2015')
plt.savefig('pci2015.png')

---

<a rel="license" href="http://creativecommons.org/licenses/by-nc-
sa/4.0/"><img alt="Creative Commons License" style="border-width:0"
src="https://i.creativecommons.org/l/by-nc-sa/4.0/88x31.png" /></a><br /><span
xmlns:dct="http://purl.org/dc/terms/" property="dct:title">This material</span> by <a xmlns:cc="http://creativecommons.org/ns#"
href="http://sergerey.org" property="cc:attributionName"
rel="cc:attributionURL">Serge Rey</a> is licensed under a <a
rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/">Creative
Commons Attribution-NonCommercial-ShareAlike 4.0 International License</a>.